In [377]:
# Para gestionar ficheros
# -----------------------------------------------------------------------
import os
import glob

# Para trabajar con access porque realmente se tendría que hacer desde la terminal pero esto nos lo simula y por eso es mejor hacerlo por aquí porque lo simplifica
# -----------------------------------------------------------------------
import subprocess


# Para trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd


# Para trabajar con regex
# -----------------------------------------------------------------------
import re

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os


# Para trabajar con los DataFrames
# -----------------------------------------------------------------------
import pandas as pd

# Para poner barras progreso
# -----------------------------------------------------------------------
from tqdm import tqdm

from datetime import datetime
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

from datetime import datetime
from dateutil.relativedelta import relativedelta

# Ignorar warings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [378]:
df_poblacional= pd.read_csv("datos/datos_descargados_INE_poblacion/03001.csv", encoding= 'latin-1', sep=';')
df_economico= pd.read_csv("datos/datos_descargados_INE_/67284.csv", encoding= 'latin-1', sep=';')

Para el futuro el id de municipio nos puede ser util asi que vamos a aprovecharlo

In [379]:
df_poblacional.sample(5)

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
7189,49 Zamora,65 y más,Extranjeros,Mujeres,2020,194
2065,39 Cantabria,0-15 años,Españoles,Hombres,2020,39.084
1830,10 Cáceres,16-64 años,% Extranjeros,Hombres,2021,"4,10"
5167,34 Palencia,65 y más,Extranjeros,Ambos sexos,2020,233
4378,28 Madrid,0-15 años,Extranjeros,Hombres,2020,70.360


In [380]:
df_economico.sample(5)

,Provincias,Ramas de actividad,periodo,Total
804,22 Huesca,F. Construcción,2021(P),442.920
282,08 Barcelona,"K_N. Actividades financieras y de seguros, act...",2021(P),42.975.113
480,12 Castellón/Castelló,"K_N. Actividades financieras y de seguros, act...",2021(P),2.748.923
1284,37 Salamanca,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2021(P),7.289.859
1472,44 Teruel,"K_N. Actividades financieras y de seguros, act...",2019,537.641


**Demanda Eléctrica:**

- **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales). ***DONE***

- **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes. ***DONE***

- **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas. ***DONE***

INICIO LIMPIEZA Y CARGA DE DEMANDA (Esto es para leer, limpiar y juntar los datos de demanda)

In [381]:
ruta_origen= r"C:\Users\jaime\Desktop\Thepower\Día15\laboratorio-modulo5-leccion01-etl\datos\demanda"
df_todo_demanda= pd.DataFrame()
for ccaa in os.listdir(ruta_origen):
    df=pd.read_csv(f"datos\demanda\{ccaa}", index_col=0 )

    ruta_ccaa = os.path.join(ruta_origen, ccaa)
    nombreb= ccaa.split(".csv")
    nombre_anio=nombreb[0].split("_")
    nombre_ccaa= nombre_anio[0]

    df["ccaa"]=nombre_ccaa
    df_todo_demanda= pd.concat([df_todo_demanda,df], axis=0)


In [382]:
df_todo_demanda.reset_index(drop=True, inplace=True)

Quiero conseguir el formato correcto de fecha

In [383]:
df_todo_demanda["datetime"] = df_todo_demanda["datetime"].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m'))

In [386]:
df_todo_demanda["month"] = df_todo_demanda["datetime"].apply(lambda x: pd.to_datetime(x).month)
df_todo_demanda["year"] = df_todo_demanda["datetime"].apply(lambda x: pd.to_datetime(x).year)

In [388]:
df_todo_demanda.drop(columns="datetime", inplace= True)

In [389]:
df_todo_demanda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       684 non-null    float64
 1   percentage  684 non-null    int64  
 2   cod_ccaa    684 non-null    int64  
 3   ccaa        684 non-null    object 
 4   month       684 non-null    int64  
 5   year        684 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 32.2+ KB


In [350]:
df_todo_demanda["ccaa"].unique()

array(['Andalucía', 'Aragón', 'Canarias', 'Cantabria',
       'Castilla - La Mancha', 'Castilla y León', 'Cataluña', 'Ceuta',
       'Comunidad de Madrid', 'Comunidad Foral de Navarra',
       'Comunitat Valenciana', 'Extremadura', 'Galicia', 'Illes Balears',
       'La Rioja', 'Melilla', 'País Vasco', 'Principado de Asturias',
       'Región de Murcia'], dtype=object)

In [351]:
df_todo_demanda["value"]

0      3668893.328
1      3152278.091
2      3216356.360
3      3029639.973
4      3154123.467
          ...     
679     901082.855
680     793300.950
681     707168.960
682     723781.406
683     781601.118
Name: value, Length: 684, dtype: float64

El value está en MWh

Esto se podrá utilizar en el futuro para meter los cods

In [352]:
cod_comunidades = {'Ceuta': 8744,
                        'Melilla': 8745,
                        'Andalucía': 4,
                        'Aragón': 5,
                        'Cantabria': 6,
                        'Castilla - La Mancha': 7,
                        'Castilla y León': 8,
                        'Cataluña': 9,
                        'País Vasco': 10,
                        'Principado de Asturias': 11,
                        'Comunidad de Madrid': 13,
                        'Comunidad Foral de Navarra': 14,
                        'Comunitat Valenciana': 15,
                        'Extremadura': 16,
                        'Galicia': 17,
                        'Illes Balears': 8743,
                        'Canarias': 8742,
                        'Región de Murcia': 21,
                        'La Rioja': 20}

FIN DE CREACION Y LIMPIEZA DE DEMANDA (FALTA NOMBRES COLUMNAS) y estaría bien poner el código de municipio para la futura creacion de BBDD

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

INICIO DE CARGA Y LIMPIEZA DE OFERTA 

In [390]:
ruta_origen= r"C:\Users\jaime\Desktop\Thepower\Día15\laboratorio-modulo5-leccion01-etl\datos\oferta"
df_todo_oferta= pd.DataFrame()
for ccaa in os.listdir(ruta_origen):
    df=pd.read_csv(f"datos\oferta\{ccaa}", index_col=0 )

    ruta_ccaa = os.path.join(ruta_origen, ccaa)
    nombreb= ccaa.split(".csv")
    nombre_anio=nombreb[0].split("_")
    nombre_ccaa= nombre_anio[0]

    df["ccaa"]=nombre_ccaa
    df_todo_oferta= pd.concat([df_todo_oferta,df], axis=0)

In [391]:
df_todo_oferta.reset_index(drop=True, inplace=True)

Quiero conseguir el formato correcto de fecha

In [392]:
df_todo_oferta["datetime"] = df_todo_oferta["datetime"].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m'))

In [393]:
df_todo_oferta["month"] = df_todo_oferta["datetime"].apply(lambda x: pd.to_datetime(x).month)
df_todo_oferta["year"] = df_todo_oferta["datetime"].apply(lambda x: pd.to_datetime(x).year)

In [395]:
df_todo_oferta.drop(columns="datetime", inplace= True)

KeyError: "['datetime'] not found in axis"

In [397]:
df_todo_oferta.sample(4)

,value,percentage,type,cod_ccaa,ccaa,month,year
2473,1.323001e+06,1.000000,Generación renovable,17,Galicia,5,2021
2014,3.074979e+04,0.074747,Solar fotovoltaica,15,Comunitat Valenciana,12,2021
414,3.727338e+03,0.019338,Hidroeólica,8742,Canarias,7,2019
3518,1.541308e+05,0.823251,Solar fotovoltaica,21,Región de Murcia,10,2021


Me he dado cuenta que nos da los totales metidos en Generación renovable con porcentajes 1

In [359]:
df_todo_oferta["type"].unique()

array(['Hidráulica', 'Eólica', 'Solar fotovoltaica', 'Solar térmica',
       'Otras renovables', 'Generación renovable', 'Hidroeólica',
       'Residuos renovables'], dtype=object)

In [360]:
df_todo_oferta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3557 entries, 0 to 3556
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   value       3557 non-null   float64       
 1   percentage  3557 non-null   float64       
 2   datetime    3557 non-null   datetime64[ns]
 3   type        3557 non-null   object        
 4   cod_ccaa    3557 non-null   int64         
 5   ccaa        3557 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 166.9+ KB


El value está en MWh

In [361]:
df_todo_oferta.sample(4)

,value,percentage,datetime,type,cod_ccaa,ccaa
2467,13857.7645,0.011945,2021-11-01,Residuos renovables,17,Galicia
2705,430.8380,0.004584,2019-09-01,Otras renovables,20,La Rioja
724,7105.8150,0.235056,2020-05-01,Otras renovables,6,Cantabria
1588,40801.2890,1.000000,2020-05-01,Generación renovable,13,Comunidad de Madrid


AQUÍ ACABA GENERACIÓN DE ENERGÍA (ESTÁS SIN HACER)

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

INICIO INE ECONOMICO

In [362]:
df_economico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716 entries, 0 to 1715
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Provincias          1716 non-null   object
 1   Ramas de actividad  1716 non-null   object
 2   periodo             1716 non-null   object
 3   Total               1716 non-null   object
dtypes: object(4)
memory usage: 53.8+ KB


FIN DE INE ECONOMICA

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

INICIO INE POBLACIONAL

In [363]:
df_poblacional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7632 entries, 0 to 7631
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Provincias               7632 non-null   object
 1   Edad (3 grupos de edad)  7632 non-null   object
 2   Españoles/Extranjeros    7632 non-null   object
 3   Sexo                     7632 non-null   object
 4   Año                      7632 non-null   int64 
 5   Total                    7632 non-null   object
dtypes: int64(1), object(5)
memory usage: 357.9+ KB


FIN INE POBLACIONAL